In [30]:
import pandas as pd
import numpy as np
import datetime as dt
import time
from bs4 import BeautifulSoup as bs
import requests
import os
from sqlalchemy import *
from sqlalchemy import create_engine
import calendar
import sqlite3 as sql

import sys
from IPython.display import clear_output
%matplotlib inline

def update_progress(progress, date_str):
    bar_length = 60
    
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}% Elapsed Run Time: {2}".format( "#" * block + "-" * (bar_length - block), 
                                                      progress * 100,
                                                      date_str)
    print(text)

%matplotlib inline
main_dir = os.getcwd()
module_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\SEC'
db_dir = r'C:\Users\Fang\Desktop\Python Trading\Trading\Data\DBs'

#'''
#Transaction Code Transaction
#A | Grant, award, or other acquisition of securities from the company (such as an option)
#K | Equity swaps and similar hedging transactions
#P | Purchase of securities on an exchange or from another person
#S | Sale of securities on an exchange or to another person
#D | Sale or transfer of securities back to the company
#F | Payment of exercise price or tax liability using portion of securities received from the company
#M | Exercise or conversion of derivative security received from the company (such as an option)
#G | Gift of securities by or to the insider
#V | A transaction voluntarily reported on Form 4
#J | Other (accompanied by a footnote describing the transaction)
#'''

txn_codes = """A:Grant from company
K:Equity Swaps
P:Purchase on exchange or from person
S:Sale on exchange or to person
D:Sale back to company
F:Payment of exercise price or tax liability using portion of securities received from the company
M:Exercise of derivative from company
G:Gift
V:Voluntarily transaction
J:Other""".split('\n')

txn_dict = pd.DataFrame([[t.split(':')[0],t.split(':')[1]] for t in txn_codes],
                        columns = ['txnCode','txnDesc'])

os.chdir(module_dir)
import sec_helpers
import sec_f4s

os.chdir(db_dir)

In [2]:
sec_engine = create_engine('sqlite:///SEC_F4.db', echo=False)

In [53]:
start_time = time.time()

for year in list(range(2019, 2018, -1)):
    for qtr in range(1, 5):
        
        try:
            currF4s = sec_f4s.get_sec_f4s(year,qtr)
        except:
            continue
        
        dfs = np.array_split(currF4s, 100)
        
        if year == 2020 and qtr == 1:
            dfs = dfs[10:]
        
        if year == 2019 and qtr < 3:
            continue
        
        for df in dfs:
            curr_txns = sec_helpers.create_insidertxns(df)
            curr_txns = curr_txns.merge(txn_dict,
                                        how = 'left',
                                        left_on = ['txnCode'],
                                        right_on = ['txnCode'])
            curr_txns.to_sql('InsiderTxns', 
                             con=sec_engine, 
                             if_exists='append', 
                             index_label = 'idx')
            print('{0}-Q{1}'.format(year,qtr))
            
print('Completed in {} seconds'.format(time.time() - start_time))

Progress: [############################################################] 100.0% Elapsed Run Time: 224.56 seconds for B on 2019-12-10 
Completed in 224.56244564056396 seconds
2019-Q4
Completed in 46770.00966882706 seconds


2019